In [2]:
from sklearn.datasets import load_digits
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import cross_val_score
import numpy as np
import pandas as pd 
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB 

In [3]:
digits_data,digits_target = load_digits().data, load_digits().target 

In [319]:
cancer_data,cancer_target = load_breast_cancer().data, load_breast_cancer().target 

In [42]:
print("digits_data: mean cross_val_score for GaussianNB= {} ".format(cross_val_score(GaussianNB(),digits_data,digits_target).mean()))
print("digits_data: mean cross_val_score for MultinomialNB ={}".format(cross_val_score(MultinomialNB(),digits_data,digits_target).mean()))
print("digits_data: mean cross_val_score fro BernoulliNB ={}".format(cross_val_score(BernoulliNB(),digits_data,digits_target).mean()))

digits_data: mean cross_val_score for GaussianNB= 0.8069281956050759 
digits_data: mean cross_val_score for MultinomialNB =0.8703497369235531
digits_data: mean cross_val_score fro BernoulliNB =0.8241736304549674


In [301]:
print("cancer_data: mean cross_val_score for GaussianNB = {} ".format(cross_val_score(GaussianNB(),cancer_data,cancer_target).mean()))
print("cancer_data: mean cross_val_score for MultinomialNB ={}".format(cross_val_score(MultinomialNB(),cancer_data,cancer_target).mean()))
print("cancer_data: mean cross_val_score fro BernoulliNB ={}".format(cross_val_score(BernoulliNB(),cancer_data,cancer_target).mean()))

cancer_data: mean cross_val_score for GaussianNB = 0.9385188635305075 
cancer_data: mean cross_val_score for MultinomialNB =0.8963204471355379
cancer_data: mean cross_val_score fro BernoulliNB =0.6274181027790716


Из-за бинарной классификации у нас GaussianNB выдал max 

0.8703497369235531 max результат на датасете digits

 На вещественных признаках лучше всего сработало нормальное распределение
 Мультиномиальное распределение лучше показало себя на выборке с целыми неотрицательными значениями признаков

In [8]:
from IPython.display import Image
Image(url= "https://camo.githubusercontent.com/b69ee18a266f5834eeb60eb4e3326a0adbdbaf15/68747470733a2f2f6769746875622e636f6d2f6c6f70654c482f476175737369616e2d42617965732d636c61737369666965722f626c6f622f6d61737465722f70726f62732e504e473f7261773d74727565")

In [45]:
class gaussianNB:
    def __init__(self,*,mu = None,cov = None,n_classes = None):
        a=None
        self.mu = mu
        self.cov = cov
        self.n_classes = n_classes
    """
    @staticmethod
    def __call_probability(x,mean,var):
        
        exponent = np.exp(-(math.pow(x - mean, 2) / (2 * var)))
        return (1 / (np.sqrt(2 * np.pi * var))) * exponent
    """  
    
    def fit(self, X,y):
        self.mu = []
        self.cov = []
        
        self.n_classes = np.max(y)+1
        
        for i in range(self.n_classes):
            Xc = X[y==i]
            mu_c = np.mean(Xc, axis=0)
            self.mu.append(mu_c)
            
            cov_c = np.zeros((X.shape[1], X.shape[1]))
            for j in range(Xc.shape[0]):
                a = Xc[j].reshape((X.shape[1],1))
                b = Xc[j].reshape((1,X.shape[1]))
                cov_ci = np.multiply(a, b)
                cov_c = cov_c+cov_ci
            cov_c = cov_c/float(X.shape[0])
            self.cov.append(cov_c)
        self.mu = np.asarray(self.mu)
        self.cov = np.asarray(self.cov)

    def predict_proba(self,x):
        prob_vect = np.zeros(self.n_classes)
        
        for i in range(self.n_classes):
            mnormal = multivariate_normal(mean=self.mu[i], cov=self.cov[i],allow_singular=True)      
            prior = 1./self.n_classes
            prob_vect[i] = prior*mnormal.pdf(x)
            sumatory = 0.
            for j in range(self.n_classes):
                mnormal = multivariate_normal(mean=self.mu[j], cov=self.cov[j],allow_singular=True)
                sumatory += prior*mnormal.pdf(x)
            prob_vect[i] = prob_vect[i]/sumatory
        return prob_vect
    
    def predict(self,x):
        return np.argmax(self.predict_proba(x))
        

In [316]:
class multinomialNB:
    
    def __init__(self, classes):
        self.classes = set(classes)
        self.prob_matrix = {}
        self.prob_classes = {}
        
    def fit(self, X, y, alpha=1):

        y = np.array(y)
        X = np.array(X)
        feature_nums = len(X[0])
        sample_nums = len(X)
        classes_nums = len(self.classes)
        self.prob_matrix = {}
        self.prob_classes = {}
        for k in self.classes:
            sample_nums_k = len(y[y == k])
            #p(y)
            self.prob_classes[k] = (sample_nums_k + alpha) / (sample_nums + classes_nums * alpha)
            #p(x|y)
            self.prob_matrix[k] = (np.sum(X[y == k, :], axis=0) + alpha) / (sample_nums_k * feature_nums +  feature_nums * alpha)
        return self
        
    def _predict(self, sample):
        
        result = {}
        for k in self.classes:
            log_post_prob = np.sum(np.log(self.prob_matrix[k][sample > 0]) * sample[sample > 0]) + \
                            np.sum(np.log(1 - self.prob_matrix[k][sample == 0]))
            log_prior_prob = np.log(self.prob_classes[k])
            result[k] = log_post_prob + log_prior_prob
        return max(result, key=result.get)
    
    def predict(self, X):
        if X.ndim == 1:
            return self._predict(X)
        else:
            return np.apply_along_axis(self._predict, 1, X)
        
    

In [320]:
import math
from sklearn.metrics import accuracy_score
from scipy.stats import multivariate_normal
from scipy.stats import norminvgauss
from sklearn.model_selection import train_test_split
model = gaussianNB()
X_train, X_test, y_train, y_test = train_test_split(cancer_data,cancer_target, test_size=0.4, random_state=0)

model.fit(X_train,y_train)
hit = 0.
for i in range(X_test.shape[0]):
    ypred = model.predict(X_test[i])
    if ypred == y_test[i]:
        hit += 1

print ("modified naive Gaussian bayes score for cancer_data: "+str(hit/X_test.shape[0]))

modified naive Gaussian bayes score for cancer_data: 0.956140350877193


In [321]:
mul_model = multinomialNB(np.unique(cancer_target))
mul_model.fit(X_train,y_train,alpha=1)
predict = mul_model.predict(X_test)
print ("multinomial naive bayes score for cancer_data: "+str(accuracy_score(predict,y_test)))

multinomial naive bayes score for cancer_data: 0.36403508771929827


In [322]:
X_train, X_test, y_train, y_test = train_test_split(digits_data,digits_target, test_size=0.4, random_state=0)
mul_model = multinomialNB(np.unique(digits_target))
mul_model.fit(X_train,y_train,alpha=1)
predict = mul_model.predict(X_test)
print ("multinomial naive bayes score for cancer_data: "+str(accuracy_score(predict,y_test)))


multinomial naive bayes score for cancer_data: 0.8692628650904033
